# Get consensus peaks for `pbmc-granulocyte-sorted-3k_10x-Multiome`
Adam Klie (last updated: *09/20/2023*)
***
This notebook shows how to preprocess scATAC-seq data from `pbmc_granulocyte_sorted_3k` using the `pycisTopic` package. See https://github.com/ML4GLand/pbmc_granulocyte_sorted_3k for more details on how to download the data.

# Set-up

In [ ]:
# Load necessary packages
import os
import scanpy as sc
import pycisTopic
import pyranges as pr
import requests
import pickle
import pandas as pd
import pybiomart as pbm
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
from pycisTopic.pseudobulk_peak_calling import peak_calling
from pycisTopic.iterative_peak_calling import get_consensus_peaks

In [ ]:
# Set-up the paths to data (TODO: change to your own paths)
input_dir = '/cellar/users/aklie/data/ml4gland/pbmc_granulocyte_sorted_3k/processed/21Sep23/'
output_dir = '/cellar/users/aklie/projects/ML4GLand/use_cases/scBasset/pbmc-granulocyte-sorted-3k_10x-Multiome/processed'
tmp_dir = os.path.join(output_dir, 'tmp')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Get the fragments file mapped
fragments_dict = {'10x_pbmc': os.path.join(input_dir, 'data/pbmc_granulocyte_sorted_3k_atac_fragments.tsv.gz')}

In [ ]:
# Helpful to have
adata = sc.read_h5ad(os.path.join(input_dir, 'scRNA/adata.h5ad'))
cell_data = adata.obs
cell_data['sample_id'] = '10x_pbmc'
cell_data['celltype'] = cell_data['celltype'].astype(str) # set data type of the celltype column to str, otherwise the export_pseudobulk function will complain.
del(adata)

In [ ]:
# Get chromosome sizes (for hg38 here)
target_url='http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'
chromsizes=pd.read_csv(target_url, sep='\t', header=None)
chromsizes.columns=['Chromosome', 'End']
chromsizes['Start']=[0]*chromsizes.shape[0]
chromsizes=chromsizes.loc[:,['Chromosome', 'Start', 'End']]

# Exceptionally in this case, to agree with CellRangerARC annotations
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].replace('v', '.') for x in range(len(chromsizes['Chromosome']))]
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].split('_')[1] if len(chromsizes['Chromosome'][x].split('_')) > 1 else chromsizes['Chromosome'][x] for x in range(len(chromsizes['Chromosome']))]
chromsizes=pr.PyRanges(chromsizes)

In [ ]:
bw_paths, bed_paths = export_pseudobulk(
    input_data = cell_data,
    variable = 'celltype',                                                                     # variable by which to generate pseubulk profiles, in this case we want pseudobulks per celltype
    sample_id_col = 'sample_id',
    chromsizes = chromsizes,
    bed_path = os.path.join(output_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/'),  # specify where pseudobulk_bed_files should be stored
    bigwig_path = os.path.join(output_dir, 'scATAC/consensus_peak_calling/pseudobulk_bw_files/'),# specify where pseudobulk_bw_files should be stored
    path_to_fragments = fragments_dict,                                                        # location of fragment fiels
    n_cpu = 8,                                                                                 # specify the number of cores to use, we use ray for multi processing
    normalize_bigwig = True,
    remove_duplicates = True,
    _temp_dir = os.path.join(tmp_dir, 'ray_spill'),
    split_pattern = '-'
)

In [ ]:
# Save the paths to the bed and bigwig files
pickle.dump(bed_paths,
            open(os.path.join(output_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/bed_paths.pkl'), 'wb'))
pickle.dump(bw_paths,
           open(os.path.join(output_dir, 'scATAC/consensus_peak_calling/pseudobulk_bed_files/bw_paths.pkl'), 'wb'))

In [ ]:
# Run peak calling
macs_path='/cellar/users/aklie/opt/miniconda3/envs/scenicplus/bin/macs2'
narrow_peaks_dict = peak_calling(
    macs_path,
    bed_paths,
    os.path.join(output_dir, 'scATAC/consensus_peak_calling/MACS/'),
    genome_size='hs',
    n_cpu=8,
    input_format='BEDPE',
    shift=73,
    ext_size=146,
    keep_dup = 'all',
    q_value = 0.05,
    _temp_dir = os.path.join(tmp_dir, 'ray_spill')
)

In [ ]:
# Save the narrow peaks dictionary
pickle.dump(
    narrow_peaks_dict,
    open(os.path.join(output_dir, 'scATAC/consensus_peak_calling/MACS/narrow_peaks_dict.pkl'), 'wb')
)

In [ ]:
# Get consensus peaks
peak_half_width = 250
path_to_blacklist= os.path.join(work_dir, 'hg38-blacklist.v2.bed')
consensus_peaks = get_consensus_peaks(narrow_peaks_dict, peak_half_width, chromsizes=chromsizes, path_to_blacklist=path_to_blacklist)

In [ ]:
# Save the consensus peaks
consensus_peaks.to_bed(
    path = os.path.join(output_dir, 'scATAC/consensus_peak_calling/consensus_regions.bed'),
    keep=True,
    compression='infer',
    chain=False)

# DONE!

---